In [2]:
!pip install sentence-transformers faiss-cpu joblib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ------------------ --------------------- 8.4/18.2 MB 40.0 MB/s eta 0:00:01
   ------------------------------------ --- 16.8/18.2 MB 44.0 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 41.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------- ----- 9.7/11.3 MB 46.3 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 46.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/561.5 kB ? eta -:--:--
   ---------------------------------------- 561.5/561.5 kB ? eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 72.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.2
    Unin

In [3]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
!pip install tf-keras

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 13.4 MB/s eta 0:00:00


In [6]:
from sentence_transformers import SentenceTransformer
import faiss

In [10]:
# Thay đường dẫn dataset của bạn
dataset_path = "vihallu-warmup.csv"
df = pd.read_csv(dataset_path)

In [7]:
def split_sentences_vi(text):
    """
    Tách câu tiếng Việt bằng regex.
    """
    sentences = re.split(r'(?<=[.!?])\s+', str(text).strip())
    return [s for s in sentences if s]


In [11]:
model = SentenceTransformer("keepitreal/vietnamese-sbert")  # pretrained cho tiếng Việt

all_chunks = []
chunk_map = []  # lưu mapping chunk -> index

for i, row in df.iterrows():
    context = row["context"]
    sentences = split_sentences_vi(context)
    for sent in sentences:
        all_chunks.append(sent)
        chunk_map.append(i)

# Embed tất cả context chunks
context_embeddings = model.encode(all_chunks, convert_to_numpy=True)

# Build FAISS index
d = context_embeddings.shape[1]  # dimension
index = faiss.IndexFlatL2(d)
index.add(context_embeddings)

print(f"Indexed {len(all_chunks)} sentences from context.")

Indexed 1123 sentences from context.


In [12]:
print(df.head())

                                     id  \
0  c480e0d2-72e5-47d6-baca-a884db935c8c   
1  8e225849-4ed1-4397-8519-2b4cfee0c7c6   
2  c5a2aef8-4c3f-4bac-9fc3-814094578fc0   
3  0bae8fee-cd84-4ec1-a324-0777f6fa7a32   
4  a8dcd1ed-c9a1-4786-b97b-27244896ff95   

                                             context  \
0  Theo pháp lệnh Vincennes năm 1374, vương quốc ...   
1  Hệ thống đường biển xuất phát từ các cảng biển...   
2  Năm 1928, Bộ Giao thông khởi thảo kế hoạch côn...   
3  Sự tiến hóa của giới thực vật đã theo xu hướng...   
4  Kể từ những năm 1970, chính phủ cũng thực hiện...   

                                              prompt  \
0  Quyền phản đối được khôi phục bởi Orléans đã g...   
1  Những sông lớn nào là các tuyến đường thủy nội...   
2  Độ dài công lộ Hán Trung-Thất Bàn Quan là bao ...   
3  Vì thực vật hạt trần chiếm ưu thế trong việc t...   
4  Cacs loaji dược pham muôn vào thi trường My ph...   

                                            response      label  
0

In [14]:
query = "  Quyền phản đối được khôi phục bởi?"
query_emb = model.encode([query], convert_to_numpy=True)

k = 3
D, I = index.search(query_emb, k)
print("Top retrieved context:")
for idx in I[0]:
    print("-", all_chunks[idx])

Top retrieved context:
- Để đổi lấy sự ủng hộ của họ, Orléans cho khôi phục droit de remontrance (quyền phản đối) của Nghị viện - vốn bị Louis XIV triệt bỏ từ trước, theo đó Nghị viện có quyền phản đối những quyết định của nhà vua mà họ cho là trái với lợi ích dân tộc.
- Trong nhiều năm, người Mỹ gốc châu Phi phải đối mặt với bạo động chống lại họ nhưng họ đạt được những bước vĩ đại về công bằng xã hội qua các phán quyết của tối cao pháp viện trong đó có các vụ án như Brown đối đầu Ban Giáo dục và Loving đối đầu Virginia, Đạo luật Dân quyền 1964, Đạo luật Quyền đầu phiếu 1965, và Đạo luật Nhà ở Công bằng 1968 mà qua đó kết thúc luật Jim Crow từng hợp thức hóa tách ly chủng tộc giữa người da trắng và người da đen.
- Tên chính thức của lãnh thổ Nouvelle-Calédonie có thể được thay đổi trong tương lai gần theo hiệp nghị, theo đó "một danh xưng, một hiệu kỳ, một bài ca, một khẩu hiệu và một thiết kế tiền giấy sẽ do toàn thể các đảng phải cùng nhau tìm kiếm, nhằm thể hiện bản sắc Kanak và tư

In [15]:
X_embeddings = []
y = []

for i, row in df.iterrows():
    # Embed prompt + response
    pr_emb = model.encode(
        row["prompt"] + " " + row["response"], 
        convert_to_numpy=True
    )
    X_embeddings.append(pr_emb)
    y.append(row["label"])

X = np.array(X_embeddings)
y = np.array(y)

print("Feature shape:", X.shape)
print("Labels:", np.unique(y))


Feature shape: (198, 768)
Labels: ['extrinsic' 'intrinsic' 'no']


Data Preprocessing

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [16]:
print("Kích thước ban đầu:", df.shape)
print(df.head())

Kích thước ban đầu: (198, 5)
                                     id  \
0  c480e0d2-72e5-47d6-baca-a884db935c8c   
1  8e225849-4ed1-4397-8519-2b4cfee0c7c6   
2  c5a2aef8-4c3f-4bac-9fc3-814094578fc0   
3  0bae8fee-cd84-4ec1-a324-0777f6fa7a32   
4  a8dcd1ed-c9a1-4786-b97b-27244896ff95   

                                             context  \
0  Theo pháp lệnh Vincennes năm 1374, vương quốc ...   
1  Hệ thống đường biển xuất phát từ các cảng biển...   
2  Năm 1928, Bộ Giao thông khởi thảo kế hoạch côn...   
3  Sự tiến hóa của giới thực vật đã theo xu hướng...   
4  Kể từ những năm 1970, chính phủ cũng thực hiện...   

                                              prompt  \
0  Quyền phản đối được khôi phục bởi Orléans đã g...   
1  Những sông lớn nào là các tuyến đường thủy nội...   
2  Độ dài công lộ Hán Trung-Thất Bàn Quan là bao ...   
3  Vì thực vật hạt trần chiếm ưu thế trong việc t...   
4  Cacs loaji dược pham muôn vào thi trường My ph...   

                                      

In [18]:
df.drop(columns=["id"], inplace=True)

In [19]:
df.head()

,context,prompt,response,label
0,"Theo pháp lệnh Vincennes năm 1374, vương quốc ...",Quyền phản đối được khôi phục bởi Orléans đã g...,Quyền phản đối còn cho phép Nghị viện được bổ ...,extrinsic
1,Hệ thống đường biển xuất phát từ các cảng biển...,Những sông lớn nào là các tuyến đường thủy nội...,Các tuyến đường thủy nội địa huyết mạch chạy t...,intrinsic
2,"Năm 1928, Bộ Giao thông khởi thảo kế hoạch côn...",Độ dài công lộ Hán Trung-Thất Bàn Quan là bao ...,Công lộ Hán Trung-Thất Bàn Quan dài hơn 150 km...,extrinsic
3,Sự tiến hóa của giới thực vật đã theo xu hướng...,Vì thực vật hạt trần chiếm ưu thế trong việc t...,Thực vật hạt trần kém đa dạng hơn và hiếm gặp ...,no
4,"Kể từ những năm 1970, chính phủ cũng thực hiện...",Cacs loaji dược pham muôn vào thi trường My ph...,Các loại dược phẩm muốn vào thị trường Mỹ phải...,no


In [20]:
print("Số dòng chứa NaN:", df.isnull().sum())

Số dòng chứa NaN: context     0
prompt      0
response    0
label       0
dtype: int64


In [22]:
df["context"] = df["context"].astype(str).str.strip()
df["prompt"]  = df["prompt"].astype(str).str.strip()
df["response"] = df["response"].astype(str).str.strip()
df["label"]   = df["label"].astype(str).str.strip().str.lower()   # giả sử label là số

In [23]:
print("Các nhãn duy nhất:", df["label"].unique())
print("Phân bố nhãn:")
print(df["label"].value_counts())

Các nhãn duy nhất: ['extrinsic' 'intrinsic' 'no']
Phân bố nhãn:
label
extrinsic    66
intrinsic    66
no           66
Name: count, dtype: int64


In [33]:
# 1.5. Tách tập train/val/test
train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    stratify=df["label"], 
    random_state=42
)
train_df, val_df = train_test_split(
    train_df, 
    test_size=0.1, 
    stratify=train_df["label"], 
    random_state=42
)

print("Train:", train_df.shape)
print("Validation:", val_df.shape)
print("Test:", test_df.shape)


Train: (142, 4)
Validation: (16, 4)
Test: (40, 4)


In [35]:
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)

In [37]:
# ============================
# BƯỚC 2. Tiền xử lý văn bản (không segmentation)
# ============================
import re

def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()  # về chữ thường
    text = re.sub(r"http\S+", " ", text)  # bỏ link
    text = re.sub(r"[^0-9a-zA-ZÀ-ỹ\s]", " ", text)  # bỏ ký tự đặc biệt
    text = re.sub(r"\s+", " ", text).strip()  # xóa khoảng trắng thừa
    return text

# Áp dụng cho train/val/test
for split, df_split in zip(
    ["train", "val", "test"], 
    [train_df, val_df, test_df]
):
    df_split["context_clean"]  = df_split["context"].apply(clean_text)
    df_split["prompt_clean"]   = df_split["prompt"].apply(clean_text)
    df_split["response_clean"] = df_split["response"].apply(clean_text)

    # Lưu lại file đã tiền xử lý
    df_split.to_csv(f"{split}_clean.csv", index=False)

print("Ví dụ sau khi tiền xử lý:")
print(train_df[["response", "response_clean"]].head(10))


Ví dụ sau khi tiền xử lý:
                                              response  \
188  Ba tổ chức Việt Minh, Việt Quốc, Việt Cách thự...   
83   Ngoài Matthias Sammer và Ulf Kirsten, một số c...   
63   Các bằng chứng về lịch sử Trái Đất tại Phong N...   
85   Năm 161, hoàng đế La Mã là Traianus, người đã ...   
174  Sông Hồng và sông Đà là các tuyến đường thủy n...   
9    Kiến chúa là con kiến duy nhất chịu trách nhiệ...   
104  Vua Hán đã quyết định giải tán toàn bộ quân độ...   
18   Giới tính của Richard đã trở thành mối quan tâ...   
100  Cuộc họp của đảng Bolshevik không nhằm ủng hộ ...   
42   Sự xuất hiện của máy tính cá nhân không ảnh hư...   

                                        response_clean  
188  ba tổ chức việt minh việt quốc việt cách thực ...  
83   ngoài matthias sammer và ulf kirsten một số cầ...  
63   các bằng chứng về lịch sử trái đất tại phong n...  
85   năm 161 hoàng đế la mã là traianus người đã dẫ...  
174  sông hồng và sông đà là các tuyến đường thủy 

In [39]:
# ============================
# BƯỚC 3. Tạo Embedding
# ============================
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

# Load model đa ngôn ngữ (hỗ trợ tiếng Việt tốt)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def make_input(row):
    return f"context: {row['context_clean']} [SEP] prompt: {row['prompt_clean']} [SEP] response: {row['response_clean']}"

# Hàm tạo embedding cho một dataframe
def encode_dataframe(df, model):
    texts = df.apply(make_input, axis=1).tolist()
    embeddings = model.encode(texts, batch_size=32, show_progress_bar=True)
    return np.array(embeddings)

# Tạo embedding cho train/val/test
X_train = encode_dataframe(train_df, model)
X_val   = encode_dataframe(val_df, model)
X_test  = encode_dataframe(test_df, model)

# Lưu label (string -> giữ nguyên để xử lý sau)
y_train = train_df["label"].values
y_val   = val_df["label"].values
y_test  = test_df["label"].values

# Lưu ra file npy để dùng sau
np.save("X_train.npy", X_train)
np.save("X_val.npy", X_val)
np.save("X_test.npy", X_test)
np.save("y_train.npy", y_train)
np.save("y_val.npy", y_val)
np.save("y_test.npy", y_test)

print("X_train shape:", X_train.shape)
print("Ví dụ embedding 1 sample:", X_train[0][:10])  # in 10 giá trị đầu


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

X_train shape: (142, 384)
Ví dụ embedding 1 sample: [ 0.01581215  0.34333    -0.05604008 -0.03048981  0.11295061  0.17085275
 -0.06542513 -0.06371505  0.13694264  0.1768651 ]


In [40]:
import torch
import torch.nn as nn

class AttentionClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=128, num_heads=4, num_layers=2):
        super(AttentionClassifier, self).__init__()

        # Biến đổi input embedding thành sequence (giả sử 384 → 12 tokens, mỗi token 32-d)
        self.seq_len = 12
        self.token_dim = input_dim // self.seq_len  # 384/12 = 32
        self.proj = nn.Linear(input_dim, self.seq_len * self.token_dim)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.token_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim,
            dropout=0.3,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Classification head
        self.fc = nn.Linear(self.token_dim, num_classes)

    def forward(self, x):
        # x shape: (batch, input_dim)
        x = self.proj(x)                         # (batch, seq_len*token_dim)
        x = x.view(-1, self.seq_len, self.token_dim)  # (batch, seq_len, token_dim)

        # Transformer encoder
        x = self.transformer(x)   # (batch, seq_len, token_dim)

        # Pooling (lấy mean của các token)
        x = x.mean(dim=1)         # (batch, token_dim)

        # Classification
        out = self.fc(x)
        return out


In [44]:
# ============================
# BƯỚC 4. Train Classifier (Local)
# ============================
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# ----------------------------
# Load dữ liệu embedding
# ----------------------------
X_train = np.load("X_train.npy")
X_val   = np.load("X_val.npy")
X_test  = np.load("X_test.npy")
y_train = np.load("y_train.npy", allow_pickle=True)
y_val   = np.load("y_val.npy", allow_pickle=True)
y_test  = np.load("y_test.npy", allow_pickle=True)

# Encode label từ string -> số
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val   = le.transform(y_val)
y_test  = le.transform(y_test)

# ----------------------------
# Chuyển sang Tensor
# ----------------------------
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor   = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor   = torch.tensor(y_val, dtype=torch.long)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

# Dataset + DataLoader
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
val_dataset   = torch.utils.data.TensorDataset(X_val_tensor, y_val_tensor)
test_dataset  = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_dataset, batch_size=16)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=16)

# ----------------------------
# MLP Classifier
# ----------------------------
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# ----------------------------
# Khởi tạo model + optimizer
# ----------------------------
input_dim = X_train.shape[1]   # embedding dimension (vd: 384)
hidden_dim = 128
num_classes = len(le.classes_)

model = MLPClassifier(input_dim, hidden_dim, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Auto chọn device: GPU nếu có, không thì CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Training on:", device)
model.to(device)

# ----------------------------
# Training loop
# ----------------------------
for epoch in range(200):  # chạy 20 epochs
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            preds = outputs.argmax(dim=1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    print(f"Epoch {epoch+1}, Loss={total_loss:.4f}, Val Acc={correct/total:.4f}")

# ----------------------------
# Evaluation
# ----------------------------
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        preds = outputs.argmax(dim=1)
        y_true.extend(y_batch.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=le.classes_))


Training on: cuda
Epoch 1, Loss=9.8712, Val Acc=0.3125
Epoch 2, Loss=9.4921, Val Acc=0.5000
Epoch 3, Loss=9.0809, Val Acc=0.3750
Epoch 4, Loss=8.7036, Val Acc=0.3750
Epoch 5, Loss=8.2500, Val Acc=0.5625
Epoch 6, Loss=7.7135, Val Acc=0.5000
Epoch 7, Loss=7.2980, Val Acc=0.5000
Epoch 8, Loss=6.6060, Val Acc=0.6250
Epoch 9, Loss=6.1505, Val Acc=0.6250
Epoch 10, Loss=5.6918, Val Acc=0.6250
Epoch 11, Loss=5.2887, Val Acc=0.6250
Epoch 12, Loss=4.8855, Val Acc=0.5625
Epoch 13, Loss=4.4552, Val Acc=0.6250
Epoch 14, Loss=3.9488, Val Acc=0.5625
Epoch 15, Loss=3.7368, Val Acc=0.5625
Epoch 16, Loss=3.3647, Val Acc=0.5625
Epoch 17, Loss=2.9746, Val Acc=0.5625
Epoch 18, Loss=2.7572, Val Acc=0.6250
Epoch 19, Loss=2.4858, Val Acc=0.5625
Epoch 20, Loss=2.4000, Val Acc=0.6250
Epoch 21, Loss=2.0085, Val Acc=0.5625
Epoch 22, Loss=1.9529, Val Acc=0.6250
Epoch 23, Loss=1.5743, Val Acc=0.6250
Epoch 24, Loss=1.5187, Val Acc=0.6250
Epoch 25, Loss=1.4120, Val Acc=0.6250
Epoch 26, Loss=1.3999, Val Acc=0.6250
Epo